In [4]:
import torch
import pandas as pd
import pyaging as pya
import os

In [ ]:
os.system("curl -o coefficients.xlsx https://elifesciences.org/download/aHR0cHM6Ly9jZG4uZWxpZmVzY2llbmNlcy5vcmcvYXJ0aWNsZXMvNDA2NzUvZWxpZmUtNDA2NzUtc3VwcDMtdjIueGxzeA--/elife-40675-supp3-v2.xlsx?_hash=qzOMc4yUFACfDFG%2FlgxkFTHWt%2BSXSmP9zz1BM3oOTRM%3D")

In [8]:
df = pd.read_excel('coefficients.xlsx', sheet_name='Blood', nrows=90)

intercept = 0.0

df['feature'] = df['Chromosome'].astype(str) + ':' + df['Position'].astype(int).astype(str)
df['coefficient'] = df['Weight']

In [9]:
features = df['feature'].tolist()

weights = torch.tensor(df['coefficient'].tolist()).unsqueeze(0)
intercept = torch.tensor([intercept])

In [10]:
model = pya.models.LinearModel(len(features))

model.linear.weight.data = weights
model.linear.bias.data = intercept

model

LinearModel(
  (linear): Linear(in_features=90, out_features=1, bias=True)
)

In [40]:
weights_dict = {
    'preprocessing': None, 
    'preprocessing_helper': None,
    'postprocessing': "petkovichblood",
    'postprocessing_helper': None,
    'features': features,
    'weight_dict': model.state_dict(),
}

metadata_dict = {
    'species': 'Mus musculus',
    'data_type': 'methylation',
    'year': 2017,
    'implementation_approved_by_author(s)': '⌛',
    'preprocessing': weights_dict['preprocessing'], 
    'postprocessing': weights_dict['postprocessing'], 
    'citation': "Petkovich, Daniel A., et al. \"Using DNA methylation profiling to evaluate biological age and longevity interventions.\" Cell metabolism 25.4 (2017): 954-960.",
    'doi': "https://doi.org/10.1016/j.cmet.2017.03.016",
    "notes": None,
}

In [41]:
torch.save(weights_dict, '../weights/petkovichblood.pt')
torch.save(metadata_dict, '../metadata/petkovichblood.pt')